In [1085]:
import pandas as pd
import requests
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import islice
from os.path import exists
import requests
import traceback

pd.set_option('display.max_colwidth', None)

In [1355]:
class MyPortfolio:
    """
    Python ETL script for Friktion user portfolio data. 
    
    Currently supported Instruction Names: 
        - Deposit
        - CancelPendingDeposit
        - Withdrawal
        - CancelPendingWithdrawal
        - ClaimPendingWithdrawal
    """
    
    
    def __init__(self, 
                 date_start, 
                 date_end, 
                 ix_fname='friktion_ix.csv', 
                 deposit_fname='friktion_deposit.csv', 
                 deposit_cxl_fname='friktion_deposit_cxl.csv', 
                 withdraw_fname='friktion_withdraw.csv', 
                 withdraw_cxl_fname='friktion_withdraw_cancel.csv',
                 withdraw_claim_fname='friktion_claim_withdrawal.csv',
                 batch_size_days=14, 
                 batch_size_xfers=75
            ):
        """
        :ix_fname:              output csv for instructions
        :deposit_fname:         output csv for deposits
        :deposit_cxl_fname:     output csv for deposit cancels
        :withdraw_fname:        output csv for withdrawals
        :withdraw_cxl_fname:    output csv for withdrawal cancels
        :withdraw_claim_fname:    output csv for claiming pending withdrawal
        :batch_size_days:       batch size in days for query to keep query < 10k rows. Use bigger steps for larger data.
        :batch_size_transfers:  batch size transactions for query to keep query < 8kb 

        """
        self.volt_program = "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"
        self.date_start = date_start
        self.date_end = date_end
        self.ix_fname = ix_fname
        self.deposit_fname = deposit_fname
        self.deposit_cxl_fname = deposit_cxl_fname
        self.withdraw_fname = withdraw_fname
        self.withdraw_cxl_fname = withdraw_cxl_fname
        self.withdraw_claim_fname = withdraw_claim_fname
        self.batch_size_days = batch_size_days
        self.batch_size_xfers = batch_size_xfers
        self.df_ix = []
        self.friktion_metadata = self.get_friktion_snapshot()


    ########################################################################################################
    ####################################          Queries             ######################################
    ########################################################################################################
    
    
    @property
    def ix_query(self):
        return """
            query MyQuery {
              solana {
                instructions(
                  time: {between: ["%s", "%s"]}
                  success: {is: true}
                  programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
                ) {
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  log {
                    consumed
                    logs
                  }
                  transaction {
                    signature
                    success
                    feePayer
                  }
                  data {
                    base58
                  }
                }
              }
            }
        """
        
        
    @property
    def xfer_query(self):
        return """
            query MyQuery {
              solana(network: solana) {
                transfers(
                  signature: {in: [%s]}
                ) {
                  instruction {
                    action {
                      name
                      type
                    }
                    callPath
                  }
                  amount(success: {is: true})
                  transaction {
                    signer
                    signature
                  }
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  currency {
                    name
                    address
                    symbol
                    decimals
                  }
                  sender {
                    address
                    mintAccount
                  }
                  receiver {
                    address
                    mintAccount
                  }
                }
              }
            }
        """
        
        
    ########################################################################################################
    ################################          Helper Functions             #################################
    ########################################################################################################
    
    
    # TODO: Add retry logic to this in case of hangups. 
    @staticmethod
    def run_query(query):  # A simple function to use requests.post to make the API call.
        headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
        request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
        if request.status_code == 200:
            return request.json()
        else:
            print(request.reason)
            raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))
    
    
    @staticmethod
    def batch_iterable(iterable, n=1):
        """
        Takes in an iterable and returns an iterable of iterables with len==x
        """
        idxs = []
        l = len(iterable)
        for idx in range(0, l, n):
            idxs.append(iterable[idx:min(idx+n, l)])
        return idxs
        
        
    def format_txs_for_query(self, tx_signatures):
        """
        Batches a list of transactions into a list of string formatted transactions for querying. 
        Each of these strings contain (n=self.batch_size_xfers) unique transaction IDs.
        """
        batched_signatures = self.batch_iterable(tx_signatures, self.batch_size_xfers)
        
        def format_txs(x):
            return str(x)[1:-1].replace("\'", "\"").replace("\n", "")
        
        tx_strs = list(map(format_txs, batched_signatures))

        return tx_strs
    
    
    def get_existing_df(self, fname):
        # Create output file if doesn't exist
        if fname and exists(fname):
            return pd.read_csv(fname)
        else:
            return pd.DataFrame()
    
    @staticmethod
    def instruction_match(instructionData):
        if not instructionData or len(instructionData) < 8:
            return False
        
        instructionDescriptor = instructionData[:8]
        
        if instructionDescriptor == "PcB3tF1K":
            return "Withdraw"
        elif instructionDescriptor == "WuE7Hjns":
            return "Deposit"
        elif instructionDescriptor == "V8cW2nMq":
            return "CancelPendingDeposit"
        elif instructionDescriptor == "dxUbSCWk":
            return "CancelPendingWithdrawal"
        elif instructionDescriptor == "WcTWQsnk":
            return "ClaimPendingWithdrawal"
        else:
            return "Unclassified"
        
        
    def get_friktion_snapshot(self):
        """
        Load Friktion Metadata for Volt/Symbol Mapping to join to normal data
        
        """
        try:
            return pd.DataFrame(
                dict(
                    json.loads(
                        requests.get("https://friktion-labs.github.io/mainnet-tvl-snapshots/friktionSnapshot.json"
                        ).content)
                  )['allMainnetVolts']
            )[["globalId", "vaultAuthority", "shareTokenMint", "depositTokenSymbol", "depositTokenCoingeckoId"]]
        except Exception as e:
            print(datetime.now(), "Snapshot Data Invalid")
            traceback.print_exc()
    
            
    ########################################################################################################
    ################################          Data Retrieval             ###################################
    ########################################################################################################
    
    
    def get_ix(self, date_start, date_end):
        """
        Runs graphql instruction query for one date range. 
        """
        query = self.ix_query % (date_start, date_end)
        print(datetime.now(), "retrieving instructions for {} to {}".format(date_start, date_end))
        result = self.run_query(query)
        
        # convert GraphQL json to pandas dataframe
        df = pd.json_normalize(result['data']['solana']['instructions'])
        print(datetime.now(), df.shape[0], "instructions retrieved")
        
        df = df.rename(
            columns={
                "block.timestamp.iso8601": "timestamp", 
                "log.consumed": "computeUnits", 
                "log.logs": "programLogs", 
                "transaction.signature": "txSignature", 
                "transaction.success": "txSuccess", 
                "transaction.feePayer": "txSigner",
                "data.base58": "instructionData"
            }
        )
        return df
    
    
    def get_ix_batch(self):
        """
        Batch the instruction retrieval. Save the shit Drop duplicates. 

        """
                
        # Batch the days up nice and good so the graphql API calls don't bitch
        dates_batched = pd.date_range(self.date_start, self.date_end, freq='7D')
        dates_batched = [str(x.isoformat()) for x in dates_batched.append(pd.DatetimeIndex([self.date_end]))]
        date_ranges = list(zip(dates_batched, dates_batched[1:]))
        
        ixs = []
        
        for date_range in date_ranges:
            assert len(date_range)==2
            data = self.get_ix(date_range[0], date_range[1])
            ixs.append(data)
            
        df_ix = pd.concat(ixs, ignore_index=False)
        df_ix["instructionType"] = df_ix.instructionData.apply(lambda x: self.instruction_match(x))
        
        # Store df_ix in self before we write it to the DataFrame
        self.df_ix = df_ix.drop_duplicates()
        print(datetime.now(), "final instruction data size: ", df_ix.shape[0])

        df_old = self.get_existing_df(self.ix_fname)
        df_ix.drop_duplicates()
        
        df = pd.concat([df_old, df_ix], ignore_index=True)
        df.to_csv(self.ix_fname, index=False)
        print(datetime.now(), "wrote instruction data to csv...")

        
        
    def get_batched_xfers(self, instructionType, fname):
        """
        Get all transfers corresponding to a specific instructionType from Graphql query. 
        Batch these queries up b/c the string sizes are too large (curse GraphQL for not supporting joins)
        
        :instructionType: String corresponding to the instruction type of each query. 
        :fname: Name of where the old df is stored
        
        """
        # assert self.df_ix, "Error: instructions get_ix_batch() must be called before xfers are scraped"
            
        temp = self.df_ix.query("instructionType == '%s'" % (instructionType))
        
        if temp.empty:
            print(datetime.now(), "instructionType was not found in the data... breaking")
            return
        
        tx_signatures = list(temp["txSignature"].unique())
        tx_strs = self.format_txs_for_query(tx_signatures)
        print(datetime.now(), len(tx_strs), "signature batches required...")
        xfers = []
        
        for i, tx_str in enumerate(tx_strs):
            query = self.xfer_query % (tx_str)
            result = self.run_query(query)
            df = pd.json_normalize(result['data']['solana']['transfers'])
            xfers.append(df)
            print(datetime.now(), df.shape[0], "transfers scraped in batch %d" % i)

        df_xfer = pd.concat(xfers, ignore_index=False)
        df_xfer = df_xfer.rename(
            columns={
                "block.timestamp.iso8601": "timestamp", 
                "instruction.action.name": "instructionAction", 
                "instruction.callPath": "instructionOrder", 
                "transaction.signer": "userAddress", 
                "transaction.signature": "txSignature", 
                "currency.symbol": "currencySymbol", 
                "currency.name": "currencyName", 
                "receiver.address": "receiverAddress", 
                "sender.address": "senderAddress", 
                "currency.decimals": "currencyDecimals",
                "currency.address": "currencyAddress", 
                "sender.mintAccount": "senderTokenMint"
            }
          )
        
        print(datetime.now(), df_xfer.shape[0], "transfers retrieved")

        df_old = self.get_existing_df(fname)
        df_final = df_old.append(df_xfer, ignore_index=True).sort_values("instructionOrder")
        
        return df_final
    

    def parse_deposits(self):
        instructionType = 'Deposit'
        instructionAction = "transfer"
        tx_merge_key = "receiverAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.deposit_fname
        
        self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)
        
            

    def parse_withdrawal(self):
        instructionType = 'Withdraw'
        instructionAction = "transfer"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_fname
        
        self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)
        
        
        
    def parse_deposit_cancel(self):
        instructionType = 'CancelPendingDeposit'
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.withdraw_fname
        
        self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)
                
        
    def parse_withdrawal_cancel(self):
        instructionType = 'CancelPendingWithdrawal'
        instructionAction = "mintTo"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_cxl_fname
        
        self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)

        
    def parse_claim_withdrawal(self):
        instructionType = 'ClaimPendingWithdrawal'
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.withdraw_claim_fname

        self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)

        
    def parse_base(self, instructionType, instructionAction, tx_merge_key, meta_merge_key, output_file):
        """
        generalized method for parsing transfer data. 
        
        1. Call get_batched_xfers()
        2. for each unique txSignature, find the xfer matching to the last instance of the instructionAction
        3. Join it to the friktion metadata based using tx_merge_key and meta_merge_key
        4. Drop extraneous rows
        5. Save the file to the output_file
        
        :instructionType: Type of instruction listed out in the instruction_match() method
        :instructionAction: type of transfer we are matching towards
        :tx_merge_key: what key in the xfer dataFrame do we want to merge on
        :meta_merge_key: what key in the metadata dataFrame we want to merge on. 
        :output_file: as name suggests
        """
        df = self.get_batched_xfers(instructionType, output_file)
        
        # Get rid of confusing wSOL entries
        df = df.query('currencyName != "Wrapped SOL"')

        # The deposit is always the last mintTo instruction.
        df = df.query('instructionAction=="{}"'.format(instructionAction)).groupby("txSignature").last().reset_index()
        
        df = pd.merge(df, self.friktion_metadata, how='left',
                      left_on=tx_merge_key, right_on=meta_merge_key, suffixes=('', '_drop'))
        df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)
        
        df.drop_duplicates().to_csv(output_file, index=False)
        print(datetime.now(), "{} data size: {}".format(instructionType, df.shape[0]))  
        
        
    def parse_all(self):
        self.get_ix_batch()
#         self.parse_claim_withdrawal()
#         self.parse_deposit_cancel()
#         self.parse_withdrawal_cancel()
        self.parse_deposits()
        self.parse_withdrawal()

        

In [1356]:
date_start = "2022-03-27T00:00:00Z"
date_end = "2022-03-31T00:00:00Z"

In [1357]:
x = MyPortfolio(date_start, date_end)

In [1354]:
x.parse_all()

2022-03-30 23:24:29.023132 retrieving instructions for 2022-03-27T00:00:00+00:00 to 2022-03-31T00:00:00+00:00
2022-03-30 23:24:47.488019 4259 instructions retrieved
2022-03-30 23:24:47.509007 final instruction data size:  4259
2022-03-30 23:24:47.548361 wrote instruction data to csv...
2022-03-30 23:24:47.565206 44 signature batches required...
2022-03-30 23:24:55.311873 306 transfers scraped in batch 0
2022-03-30 23:24:59.340848 319 transfers scraped in batch 1
2022-03-30 23:25:09.809527 347 transfers scraped in batch 2
2022-03-30 23:25:17.812701 282 transfers scraped in batch 3
2022-03-30 23:25:24.546804 330 transfers scraped in batch 4
2022-03-30 23:25:33.040166 426 transfers scraped in batch 5
2022-03-30 23:25:41.958314 228 transfers scraped in batch 6
2022-03-30 23:25:51.183579 214 transfers scraped in batch 7
2022-03-30 23:25:57.477037 230 transfers scraped in batch 8
2022-03-30 23:26:02.087086 243 transfers scraped in batch 9
2022-03-30 23:26:10.274221 227 transfers scraped in b

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/alexdai/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_13383/3587211582.py", line 1, in <module>
    x.parse_all()
  File "/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_13383/3770460033.py", line 434, in parse_all
    self.parse_deposits()
  File "/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_13383/3770460033.py", line 352, in parse_deposits
    self.parse_base(instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file)
  File "/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_13383/3770460033.py", line 413, in parse_base
    df = self.get_batched_xfers(instructionType, output_file)
  File "/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_13383/3770460033.py", line 314, in get_batched_xfers
    result = 

TypeError: object of type 'NoneType' has no len()

In [1358]:
ix = pd.read_csv("friktion_ix.csv")

In [1359]:
ix

,timestamp,computeUnits,programLogs,txSignature,txSuccess,txSigner,instructionData,instructionType
0,2022-03-27T00:01:02Z,99312,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 99312 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3czRxfMFaTYU1aFdaDpPhGFWQW2qpWUpHu6wDj6HqgQM6xdLHhg8ykj7XuPB6Cm9KRv1LbFqzqLDU6bBnAtofnnH,True,CL2JZC5qKtJQu2fisjCRb5N83EinjwahGAUYFdQAW4GN,WuE7HjnsyeazHf1Ep7YnuD,Deposit
1,2022-03-27T00:06:14Z,85819,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 3156, deposit amount = 10, vault capacity = 1800000000;Program log: post deposit estimated underlying: 890424249, vault capacity: 1800000000;Program log: pending deposit;Program log: transferring 10 tokens from 2MtkHFTQE167G9R76KHD37fAe44wGwnopU2CLNR3ipPx to GeHRjyya4mz7msXsTGrfisMbpLCTmvasx6Q1BMbzpKtG;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 85819 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",3UX8P17fiLZRHkzNFV4Wxqjzr5B7uAVy5HVpt3xyZYLV3EeURp8bq7UWz4uuTPqerdtBkUZgkST24UeCviChWkQU,True,beer2DnvzpGjPWY5uFR729QmovL6U727mUTkLZjGtpV,WuE7HjnsyeaqDHVJYdE6Qo,Deposit
2,2022-03-27T00:10:08Z,99801,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 189274260, deposit amount = 189274260, vault capacity = 12000000000000;Program log: post deposit estimated underlying: 48361104824, vault capacity: 12000000000000;Program log: pending deposit;Program log: transferring 189274260 tokens from FjDZmWGCQ9eVWmnK8bawT1odAbxoKtWbQEgVwQrM26Tf to 9x36kfR3np8iyBCCtBQSp16gVbeQ4bXMxcBAZ3pXRdz6;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 99801 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",5U9wMy8x8LSm4vbCCmQ4JMsHJZUrsFiasN2cFLYrd4VXAzDydkkgckMCNFfQDPyXDAyEFhnSnaTQnuaL7oA8tEje,True,27sNc7WV5iciwMzainA5pngLW9CMhtWgpeae1oGAyPzF,WuE7HjnsyebEJygdbdYYbR,Deposit
3,2022-03-27T00:18:00Z,58296,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: ClaimPending;Program log: validating ClaimPending context;Program log: claim_pending deposit...;Program log: transferring w/ seeds 398877815 tokens from HQmBjXsPbroU45qFgC7zBoUfGw152PsvUAwR3Qch7Bkr to Ewi2j5Y7oPVjvfvJdkhsABjCHzUMxM72xEw88K8QADxR;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 58296 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,5A1JcdVuARZ6BrMHfA8hUgfeaT5vAPEfSB5dgexgpppV1d2qRoyd5Bax2QdBX4Zh33Ncv6RKh4RtNjwkXaa3FK2W,True,Bmj5AGcUFYYhd4XgfQ3TmZ2dmyZnss1rLAxZ69RSS7ut,YU8PpoqdnQP,Unclassified
4,2022-03-27T00:18:00Z,78853,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Withdraw;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 78853 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,5A1JcdVuARZ6BrMHfA8hUgfeaT5vAPEfSB5dgexgpppV1d2qRoyd5Bax2QdBX4Zh33Ncv6RKh4RtNjwkXaa3FK2W,True,Bmj5AGcUFYYhd4XgfQ3TmZ2dmyZnss1rLAxZ69RSS7ut,PcB3tF1KHa27oaJVPZTYL7,Withdraw
...,...,...,...,...,...,...,...,...
4254,2022-03-30T23:56:49Z,91846,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 91846 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3ktAQGwRRq8pofEDZ96GYtSp9uPNwcXhpsv5gV16w6taHqpgKf9mT79xxNrqpxf2iYxSjSJHPqnY5WW27zwRMBJX,True,HiuhBRjmFcGXcga5M5dDWvSVwcgDi17yAZS14G5rpnGh,WuE7HjnsyebTEKC789s5pT,Deposit
4255,2022-03-30T23:58:42Z,53397,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];P

In [ ]:
ix.loc[ix.instructionType=="Deposit"]